# 08-dendie-derby.ipynb

In [1]:
use_napari = True
import imageio.v3 as iio
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
if use_napari:
    import napari
import numpy as np
from pathlib import Path
from skimage import exposure, filters, registration, util
# Local imports
import helpers
# Enable autoreload for local modules
%load_ext autoreload
%autoreload 2

Note: You may need to put an "r" before the image directory path if the string contains backslashes. Don't forget commas between parameters!

In [2]:
if use_napari:
    # Open napari window
    viewer = napari.Viewer()

In [3]:
# Load images
img_dir = Path(
        r'X:\Radiography Drive\2013-11_APS'
        r'\Clarke_2013_November\057_Sn27Bi_100RB')
img_start = 112
img_stop = 352
img_step = 1
imgs_raw = helpers.get_imgs(
    img_dir,
    img_start=img_start,
    img_stop=img_stop,
    img_step=img_step,
    print_nums=False
)
if use_napari:
    viewer.add_image(imgs_raw, name=f'imgs-{img_start}-{img_stop}-{img_step}')


Loading 240 images...
Images loaded.


In [4]:
# Median filter images before converting to float
print('Applying median filter...')
imgs_med = filters.median(imgs_raw)
# Convert image to float before calculations
imgs_float = util.img_as_float(imgs_med)
# Calculate offset_1 after first movement
offset, error, diffphase = registration.phase_cross_correlation(
        imgs_float[101, :, :], imgs_float[104, :, :])
offset_1_r = int(offset[0])
offset_1_c = int(offset[1])
print(offset_1_r, offset_1_c)

Applying median filter...
374 0


In [18]:
imgs_med = None
imgs_raw = None

In [28]:
print(f'{imgs_float.shape=}')
# Calc liquid-subtracted images with offset/drift-correction
imgs_crctd = np.zeros(
        (imgs_float.shape[0],
         imgs_float.shape[1] + abs(offset_1_r),
         imgs_float.shape[2] + abs(offset_1_c)))
print(f'{imgs_crctd.shape=}')
# Add pre-offset_1 images
imgs_crctd[: 102, : imgs_float.shape[1], : imgs_float.shape[2]] = (
        imgs_float[: 102, :, :])
# Add pre-offset_1 sliver to the rest of the experiment
pre_offset_1 = [imgs_float[101, : offset_1_r, :]] * (imgs_float.shape[0] - 102)
pre_offset_1 = np.stack(pre_offset_1)
print(f'{pre_offset_1.shape=}')
imgs_crctd[102 :, : offset_1_r, :] = pre_offset_1
# Add offset_1 images
imgs_crctd[
    102 : 117,
    offset_1_r : imgs_float.shape[1] + offset_1_r,
    offset_1_c : imgs_float.shape[2] + offset_1_c
] = imgs_float[106 : 121, :, :]
if use_napari:
    viewer.add_image(imgs_crctd)

imgs_float.shape=(240, 1024, 1280)
imgs_crctd.shape=(240, 1398, 1280)
pre_offset_1.shape=(138, 374, 1280)


In [29]:
# Calculate offset_2 after first movement
offset_2, error, diffphase = registration.phase_cross_correlation(
        imgs_float[111, :, :], imgs_float[120, :, :])
offset_2_r = int(offset[0])
offset_2_c = int(offset[1])
print(offset_2_r, offset_2_c)

374 0


In [ ]:
# Process images
imgs_crctd = helpers.align_and_sub_liq(
        imgs_raw, clip=[0.1, 99.9], hist_eq_clip_lim=0.0001)
if use_napari:
    viewer.add_image(imgs_crctd)
# Save images
save_dir = Path(
    r'C:\Users\cgusb\Research\aps-directional'
    fr'\results\{img_dir.stem}-{img_start}-{img_stop}-{img_step}'
    '_med-filt_liq-sub_aligned_clip')
# Image widths and heights must be multiple of 2 for mp4 compression,
# adjust odd number width by excluding last column
helpers.save_as_pngs(
        save_dir, imgs_crctd[:, :, :-1], scalebar_dict={
                'dx' : 1.4, 'units' : "um", 'length_fraction' : 0.2,
                'border_pad' : 0.5, 'location' : 'lower right'})